## Data Retrieval Notebook

In this notebook, we use code provided by the NREL to request and handle the necessary data and metadata from the NOW-23 Great Lakes dataset. The code provided by the NREL has been modified to make the automatic querying and concatenation of data possible.

In [1]:
import requests
import pandas as pd
import numpy as np
import urllib.parse
import time
import os
from dotenv import load_dotenv, dotenv_values

load_dotenv()
API_KEY = "n3pDL9ZWwMr3JVRGqZ5oRPtb6uVZCvEqgNm6N9yg" # os.getenv("API_KEY")
EMAIL = "alexanderwycoff@icloud.com" # os.getenv("EMAIL")

In [33]:
# The function used to generate the 100 random site ids: 
# np.random.randint(0, 388080, (100))

site_ids = pd.read_csv("site ids", index_col=0)
site_ids = [str(site_ids.iloc[x, 0]) for x in range(100)]

In [55]:
BASE_URL = "https://developer.nrel.gov/api/wind-toolkit/v2/wind/offshore-great-lakes-download.csv?"
POINTS = ['141132'] # site_ids
YEARS = ['2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020']

# This code is provided by NREL to request and handle data
# Some modifications have been made so that csv requests can be made and appended automatically

def main():
    input_data = {
        'attributes': 'temperature_80m,turbulent_kinetic_energy_80m,winddirection_80m,windspeed_80m',
        'interval': '60',

        'api_key': API_KEY,
        'email': EMAIL,
    }
    
    for id, location_ids in enumerate(POINTS):

        print(f'Making request for point group {id + 1} of {len(POINTS)}...')
        for name in YEARS:
            
            input_data['names'] = name
            input_data['location_ids'] = location_ids

            if '.csv' in BASE_URL:
                url = BASE_URL + urllib.parse.urlencode(input_data, True)
                data = pd.read_csv(url, on_bad_lines='skip', skiprows=2)
                
                print(f'Response data for year {name} and site {location_ids}')

                file_loc = r"Data/NOW-23 Great Lakes [2000-2020] 60min/" + str(location_ids + ".csv")
                
                if name == '2000':
                    data.to_csv(file_loc)
                else:
                    pd.concat([pd.read_csv(file_loc, index_col=0), data]).to_csv(file_loc)
            else:
                headers = {
                    'x-api-key': API_KEY
                }
                data = get_response_json_and_handle_errors(requests.post(BASE_URL, input_data, headers=headers))
                download_url = data['outputs']['downloadUrl']
                # You can do with what you will the download url
                print(data['outputs']['message'])
                print(f"Data can be downloaded from this url when ready: {download_url}")

                data = pd.read_csv(download_url)

                # Delay for 1 second to prevent rate limiting
                time.sleep(1)
            print(f'Processed')


def get_response_json_and_handle_errors(response: requests.Response) -> dict:
    """Takes the given response and handles any errors, along with providing
    the resulting json

    Parameters
    ----------
    response : requests.Response
        The response object

    Returns
    -------
    dict
        The resulting json
    """
    if response.status_code != 200:
        print(f"An error has occurred with the server or the request. The request response code/status: {response.status_code} {response.reason}")
        print(f"The response body: {response.text}")
        exit(1)

    try:
        response_json = response.json()
    except:
        print(f"The response couldn't be parsed as JSON, likely an issue with the server, here is the text: {response.text}")
        exit(1)

    if len(response_json['errors']) > 0:
        errors = '\n'.join(response_json['errors'])
        print(f"The request errored out, here are the errors: {errors}")
        exit(1)
    return response_json

if __name__ == "__main__":
    main()

Making request for point group 1 of 1...
Response data for year 2000 and site 141132
Processed
Response data for year 2001 and site 141132
Processed
Response data for year 2002 and site 141132
Processed
Response data for year 2003 and site 141132
Processed
Response data for year 2004 and site 141132
Processed
Response data for year 2005 and site 141132
Processed
Response data for year 2006 and site 141132
Processed
Response data for year 2007 and site 141132
Processed
Response data for year 2008 and site 141132
Processed
Response data for year 2009 and site 141132
Processed
Response data for year 2010 and site 141132
Processed
Response data for year 2011 and site 141132
Processed
Response data for year 2012 and site 141132
Processed
Response data for year 2013 and site 141132
Processed
Response data for year 2014 and site 141132
Processed
Response data for year 2015 and site 141132
Processed
Response data for year 2016 and site 141132
Processed
Response data for year 2017 and site 1411

In [4]:
# This code is modified from the above cell to retrieve metadata for all randomly selected points and concatenate it

BASE_URL = "https://developer.nrel.gov/api/wind-toolkit/v2/wind/offshore-great-lakes-download.csv?"
POINTS = ['141132'] # site_ids

input_data = {
    'attributes': 'temperature_80m,turbulent_kinetic_energy_80m,winddirection_80m,windspeed_80m',
    'interval': '60',

    'api_key': API_KEY,
    'email': EMAIL,
}

for id, location_ids in enumerate(POINTS):

    print(f'Making request for point group {id + 1} of {len(POINTS)}...')
    for name in ['2000']:

        input_data['names'] = name
        input_data['location_ids'] = location_ids

        if '.csv' in BASE_URL:
            url = BASE_URL + urllib.parse.urlencode(input_data, True)
            data = pd.read_csv(url, on_bad_lines='skip')

            print(f'Response metadata for year {name} and site {location_ids}')
            
            file_loc = r"Data/Muskegon site metadata.csv"
            if id == 0:
                data.to_csv(file_loc)
            else:
                pd.concat([pd.read_csv(file_loc, index_col=0), data]).to_csv(file_loc)

Making request for point group 1 of 1...
Response metadata for year 2000 and site 141132


In [38]:
import requests
import pandas as pd
import urllib.parse
import time

BASE_URL = "https://developer.nrel.gov/api/wind-toolkit/v2/wind/wtk-led-conus-download.csv?"
POINTS = site_ids

def main():
    input_data = {
        'attributes': 'temperature_80m,winddirection_80m,windspeed_80m,vertical_windspeed_80m,virtual_potential_temperature_80m',
        'interval': '60',

        'api_key': API_KEY,
        'email': EMAIL,
    }
    for name in ['2018','2019','2020']:
        print(f"Processing name: {name}")
        for id, location_ids in enumerate(POINTS):
            input_data['names'] = [name]
            input_data['location_ids'] = location_ids
            print(f'Making request for point group {id + 1} of {len(POINTS)}...')

            if '.csv' in BASE_URL:
                url = BASE_URL + urllib.parse.urlencode(input_data, True)
                data = pd.read_csv(url, on_bad_lines='skip', skiprows=1)

                print(f'Response data for year {name} and site {location_ids}')

                file_loc = r"Data\WTK_LED CONUS [2018-2020] 60min/" + str(location_ids + ".csv")

                if name == '2018':
                    data.to_csv(file_loc)
                else:
                    pd.concat([pd.read_csv(file_loc, index_col=0), data]).to_csv(file_loc)
            else:
                headers = {
                    'x-api-key': API_KEY
                }
                data = get_response_json_and_handle_errors(requests.post(BASE_URL, input_data, headers=headers))
                download_url = data['outputs']['downloadUrl']
                # You can do with what you will the download url
                print(data['outputs']['message'])
                print(f"Data can be downloaded from this url when ready: {download_url}")

                # Delay for 1 second to prevent rate limiting
                time.sleep(1)
            print(f'Processed')


def get_response_json_and_handle_errors(response: requests.Response) -> dict:
    """Takes the given response and handles any errors, along with providing
    the resulting json

    Parameters
    ----------
    response : requests.Response
        The response object

    Returns
    -------
    dict
        The resulting json
    """
    if response.status_code != 200:
        print(f"An error has occurred with the server or the request. The request response code/status: {response.status_code} {response.reason}")
        print(f"The response body: {response.text}")
        exit(1)

    try:
        response_json = response.json()
    except:
        print(f"The response couldn't be parsed as JSON, likely an issue with the server, here is the text: {response.text}")
        exit(1)

    if len(response_json['errors']) > 0:
        errors = '\n'.join(response_json['errors'])
        print(f"The request errored out, here are the errors: {errors}")
        exit(1)
    return response_json

if __name__ == "__main__":
    main()

Processing name: 2018
Making request for point group 1 of 100...
Response data for year 2018 and site 244138
Processed
Making request for point group 2 of 100...
Response data for year 2018 and site 30891
Processed
Making request for point group 3 of 100...
Response data for year 2018 and site 56031
Processed
Making request for point group 4 of 100...
Response data for year 2018 and site 58085
Processed
Making request for point group 5 of 100...
Response data for year 2018 and site 282816
Processed
Making request for point group 6 of 100...
Response data for year 2018 and site 118158
Processed
Making request for point group 7 of 100...
Response data for year 2018 and site 88246
Processed
Making request for point group 8 of 100...
Response data for year 2018 and site 178499
Processed
Making request for point group 9 of 100...
Response data for year 2018 and site 326920
Processed
Making request for point group 10 of 100...
Response data for year 2018 and site 164677
Processed
Making requ

In [6]:
for filename in os.listdir("Data/WTK_LED CONUS [2018-2020] 60min"):
    # turbulent kinetic energy at 80m (m2/s2)
    wtk_data = pd.read_csv("Data/WTK_LED CONUS [2018-2020] 60min/" + filename, index_col=0).reset_index(drop=True)
    now_data = pd.read_csv(r"Data/NOW-23 Great Lakes [2000-2020] 60min/" + filename, index_col=0)
    wtk_data['wind direction at 80m (deg)'] = list(now_data[now_data['Year'] >= 2018]['wind direction at 80m (deg)'])
    wtk_data['wind speed at 80m (m/s)'] = list(now_data[now_data['Year'] >= 2018]['wind speed at 80m (m/s)'])
    wtk_data.to_csv("Data/WTK_LED CONUS [2018-2020] 60min/" + filename)

In [3]:
year = "2013"
data = pd.read_csv(f"Data/lidar data/buoy{year}.csv")
new_data = pd.DataFrame(columns = ["Year", "Month", "Day", "Hour", "Minute", "wind direction at 80m (deg)", "wind speed at 80m (m/s)"])
for i in range(len(data)):
    time = data['DataTimeStamp'].iloc[i]
    firstslash = time.find("/")
    secondslash = time[firstslash + 1:].find("/") + firstslash + 1
    space = secondslash + 3
    colon = time.find(":")
    new_data.loc[len(new_data)] = [year, time[:firstslash], time[firstslash + 1:secondslash], time[space + 1:colon], time[colon + 1:], data.iloc[i, 8], data.iloc[i, 2]]

/var/folders/3n/_v3ftgwd5b782s60l1_nwtbm0000gn/T/ipykernel_81117/1211817290.py:2: DtypeWarning: Columns (1,26,27,28,29,30,31,32,34,35,36,37) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(f"Data/lidar data/buoy{year}.csv")


In [17]:
data = new_data
start_index = data[(data['Year'] == "2013") & (data['Month'] == "11") & (data["Day"] == "20")].index[0]
end_index = data[(data['Year'] == "2013") & (data['Month'] == "12") & (data["Day"] == "20")].index[-1]
data = data[start_index : end_index]
data = data.reset
lastval = 0
num_missing = 0
for column in ["wind speed at 80m (m/s)"]:
    for i in range(len(data)):
        try:
            value = float(data.loc[i, column])
            if np.isnan(value):
                data.loc[i, column] = data.loc[lastval, column]
                num_missing += 1
            else:
                lastval = i
        except:
            data.loc[i, column] = data.loc[lastval, column]
            num_missing += 1

print(num_missing, len(data), num_missing / (len(data)))
# data.to_csv(f"Data/lidar data/buoy{year}compiled.csv")

,Year,Month,Day,Hour,Minute,wind direction at 80m (deg),wind speed at 80m (m/s)
29670,2013,11,20,0,00,140.4,5
29671,2013,11,20,0,10,141.3,5.3
29672,2013,11,20,0,20,136.9,5.3
29673,2013,11,20,0,30,140,5.5
29674,2013,11,20,0,40,141.7,6.1
...,...,...,...,...,...,...,...
34129,2013,12,20,23,00,60.7,7.5
34130,2013,12,20,23,10,59.9,7.1
34131,2013,12,20,23,20,62.3,7.4
34132,2013,12,20,23,30,65.5,7.2


0


KeyError: 0